# 01 – Exploratory Data Analysis: Cash Conversion Cycle (CCC)

This notebook explores synthetic and raw data to compute:
- Days Sales Outstanding (DSO)
- Days Payables Outstanding (DPO)
- Days Inventory Outstanding (DIO)
- Cash Conversion Cycle (CCC)

The goal is to **quantify working-capital efficiency** and identify improvement levers.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from value_ops_lab.synth import make_synthetic
from value_ops_lab.ccc import dso, dpo, dio, ccc

# Generate synthetic dataset
df = make_synthetic(n_months=24)

# Build inputs
sales_df = df[["month","sales"]]
cogs_df = df[["month","cogs"]]

dso_df = dso(df[["month","ar_balance"]], sales_df)
dpo_df = dpo(df[["month","ap_balance"]], cogs_df)
dio_df = dio(df[["month","inventory"]], cogs_df)

ccc_df = ccc(dso_df, dpo_df, dio_df)
ccc_df.head()

ModuleNotFoundError: No module named 'value_ops_lab'

In [ ]:
# Plot CCC over time
plt.figure(figsize=(10,5))
plt.plot(ccc_df["month"], ccc_df["CCC"], label="CCC")
plt.title("Cash Conversion Cycle Over Time")
plt.ylabel("Days")
plt.legend()
plt.show()

### Key Insight
- High CCC → more cash tied up.
- Improving DSO/DPO/DIO can unlock liquidity.